<a href="https://colab.research.google.com/github/DiegoMeloCoder/Machine_Learning_T3Profundizacion/blob/main/ModeloLogistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carga de datos

In [ ]:
from os import read
import sqlite3
import pandas as pd

# Conexion
conexion = sqlite3.connect('datos_sensores.db')

# Codigo sql para hacer consulta
query = """

SELECT
    datos_basicos.tamaño,

    orion.sensor_orion,
    vega.sensor_vega,
    polaris.sensor_polaris,
    antares.sensor_antares,
    clasificacion.id,
    CASE
        WHEN clasificacion.etiqueta = 'Positivo' THEN 1
        ELSE 0
    END as segmento_numerico
FROM datos_basicos
LEFT JOIN orion ON datos_basicos.id = orion.id
LEFT JOIN vega ON datos_basicos.id = vega.id
LEFT JOIN polaris ON datos_basicos.id = polaris.id
LEFT JOIN antares ON datos_basicos.id = antares.id
LEFT JOIN clasificacion ON datos_basicos.id = clasificacion.id
WHERE clasificacion.etiqueta IS NOT NULL AND clasificacion.etiqueta <> '';

"""

dataset = pd.read_sql_query(query,conexion)
dataset

,tamaño,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,id,segmento_numerico
0,0.02,19.77,42.15,35.36,-62.73,1,1
1,0.11,-23.51,-20.64,69.78,-38.02,2,1
2,0.85,86.63,75.26,30.82,-71.62,3,0
3,0.53,-44.91,4.11,71.92,27.17,4,0
4,0.14,-66.34,-48.04,-2.91,-74.93,5,0
...,...,...,...,...,...,...,...
299995,0.30,-81.98,20.64,8.07,40.18,299996,1
299996,0.21,-2.74,-81.87,-15.87,40.64,299997,1
299997,0.12,-13.86,-27.85,63.55,19.37,299998,0
299998,0.09,-3.87,-69.29,86.51,-15.73,299999,1


creacion y entrenamiento


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = dataset.drop(['segmento_numerico', 'id'],axis=1)
Y= dataset['segmento_numerico']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=10)

#Creacion del Modelo
modelo = LogisticRegression(max_iter=100000000)
modelo.fit(X_train,Y_train)

LogisticRegression(max_iter=100000000)

Test

In [ ]:
prediccones = modelo.predict(X_test)

accuracy = accuracy_score(Y_test,prediccones)
print(f"precision del modelo: {accuracy:.2f}")

precision del modelo: 0.52


In [ ]:
reporte = classification_report(Y_test, prediccones)
print(reporte)

              precision    recall  f1-score   support

           0       0.50      0.90      0.64     22649
           1       0.49      0.10      0.16     22351

    accuracy                           0.50     45000
   macro avg       0.50      0.50      0.40     45000
weighted avg       0.50      0.50      0.41     45000



In [ ]:
matriz_confusion = confusion_matrix(Y_test, prediccones)
print(matriz_confusion)

[[20577  2102]
 [20365  1956]]


Con el modelo ya entrenado vamos a rellenar los datos faltantes


In [ ]:
 # query que selecciona los datos faltantes

 query2 = """

SELECT
datos_basicos.tamaño,
    datos_basicos.categoria,
    orion.sensor_orion,
    vega.sensor_vega,
    polaris.sensor_polaris,
    antares.sensor_antares,
    clasificacion.id,
    CASE
        WHEN clasificacion.etiqueta = 'Positivo' THEN 1
        ELSE 0
    END as segmento_numerico
	FROM datos_basicos
LEFT JOIN orion ON datos_basicos.id = orion.id
LEFT JOIN vega ON datos_basicos.id = vega.id
LEFT JOIN polaris ON datos_basicos.id = polaris.id
LEFT JOIN antares ON datos_basicos.id = antares.id
LEFT JOIN clasificacion ON datos_basicos.id = clasificacion.id
WHERE clasificacion.etiqueta IS NULL OR clasificacion.etiqueta = '';

 """

 dataset2 = pd.read_sql_query(query2,conexion)

# Predictores de las variables a rellenar
X_completar = dataset2.drop(['segmento_numerico', 'id'],axis=1)


,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares
0,0.10,2,-91.99,90.35,12.85,45.19
1,0.93,2,2.98,14.86,-22.14,45.21
2,0.86,3,-22.53,10.25,74.24,1.93
3,0.86,1,24.14,-31.35,-67.50,29.67
4,0.92,3,16.14,98.17,-14.76,10.59
...,...,...,...,...,...,...
95,0.61,3,75.72,-65.18,75.00,-33.67
96,0.08,1,-3.33,-28.75,-97.28,30.82
97,0.41,3,-48.04,-23.84,32.14,-46.10
98,0.71,1,15.46,-39.97,45.71,64.84


Uso del modelo anteriormente entrenado para predecir datos faltantes.

In [ ]:
datos_faltantes = modelo.predict(X_completar)
datos_faltantes


array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])

Dataset con id y predicciones

In [ ]:
# Tabla completa
